# Trajectory picture

Note: Running Simulation and Digitization in the same notebook is fragile. Restart after each and only run the specific cells.

In [1]:
import math
import os
import subprocess

import joblib
import matplotlib
import palettable
import ROOT

Welcome to JupyROOT 6.16/00


In [2]:
def simulation():
    parfile = "output/traj.para.root"
    outfile = "output/traj.simu.root"

    ROOT.ROOT.EnableThreadSafety()
    ROOT.FairLogger.GetLogger().SetLogVerbosityLevel("LOW")
    ROOT.FairLogger.GetLogger().SetLogScreenLevel("ERROR")

    vmcworkdir = os.environ["VMCWORKDIR"]
    os.environ["GEOMPATH"] = vmcworkdir + "/geometry"
    os.environ["CONFIG_DIR"] = vmcworkdir + "/gconfig"
    os.environ["PHYSICSLIST"] = "QGSP_INCLXX_HP"

    # Initialize Simulation
    run = ROOT.FairRunSim()
    run.SetName("TGeant4")
    run.SetStoreTraj(True)
    run.SetMaterials("media_r3b.geo")

    # Output
    output = ROOT.FairRootFileSink(outfile)
    run.SetSink(output)

    # Primary Generator
    generator = ROOT.FairPrimaryGenerator()
    box = ROOT.FairBoxGenerator(2112, 1)
    box.SetXYZ(0, 0, 0.0)
    box.SetThetaRange(0.0, 1.0)
    box.SetPhiRange(0.0, 360.0)
    box.SetEkinRange(0.6, 0.6)
    generator.AddGenerator(box)
    run.SetGenerator(generator)

    # Geometry
    cave = ROOT.R3BCave("Cave")
    cave.SetGeometryFileName("r3b_cave_vacuum.geo")
    run.AddModule(cave)

    doubleplane = 30
    neuland_position = ROOT.TGeoTranslation(0.0, 0.0, 15 * 100 + doubleplane * 10.0 / 2.0)
    neuland = ROOT.R3BNeuland(doubleplane, neuland_position)
    run.AddModule(neuland)

    # Prepare to run
    run.Init()
    ROOT.TVirtualMC.GetMC().SetRandom(ROOT.TRandom3(1337))
    ROOT.TVirtualMC.GetMC().SetMaxNStep(100000)

    # Runtime Database
    rtdb = run.GetRuntimeDb()
    parout = ROOT.FairParRootFileIo(True)
    parout.open(parfile)
    rtdb.setOutput(parout)
    rtdb.saveOutput()

    run.Run(40)

In [3]:
# simulation()

In [4]:
def digitization():
    inpfile = "output/traj.simu.root"
    parfile = "output/traj.para.root"
    outfile = "output/traj.digi.root"

    if not os.path.isfile(inpfile):
        print(f"Input {inpfile} does not exist")
        return

    ROOT.ROOT.EnableThreadSafety()
    ROOT.FairLogger.GetLogger().SetLogVerbosityLevel("LOW")
    ROOT.FairLogger.GetLogger().SetLogScreenLevel("WARNING")

    run = ROOT.FairRunAna()
    run.SetSource(ROOT.FairFileSource(inpfile))
    run.SetSink(ROOT.FairRootFileSink(outfile))

    # Connect Runtime Database
    rtdb = run.GetRuntimeDb()
    pario = ROOT.FairParRootFileIo(False)
    pario.open(parfile)
    rtdb.setFirstInput(pario)
    rtdb.setOutput(pario)
    rtdb.saveOutput()

    # Digitize data to hit level and create respective histograms
    run.AddTask(ROOT.R3BNeulandDigitizer())

    # Build clusters and create respective histograms
    run.AddTask(ROOT.R3BNeulandClusterFinder())

    # Find the actual primary interaction points and their clusters
    run.AddTask(ROOT.R3BNeulandPrimaryInteractionFinder())
    run.AddTask(ROOT.R3BNeulandPrimaryClusterFinder())

    # Create spectra
    run.AddTask(ROOT.R3BNeulandMCMon())
    run.AddTask(ROOT.R3BNeulandHitMon())
    run.AddTask(ROOT.R3BNeulandClusterMon("NeulandPrimaryClusters", "NeulandPrimaryClusterMon"))
    run.AddTask(ROOT.R3BNeulandClusterMon())

    run.Init()
    run.Run(0, 0)

In [5]:
# Note: Will kill the worker if simulation ran
# digitization()

In [6]:
f = ROOT.TFile.Open("output/traj.simu.root")
t = f.evt
t.AddFriend("evt", "output/traj.digi.root")

zoffset = 1500

allparticles = []
allpoints = []
allhits = []
for e in t:
    data = []
    for i, track in enumerate(e.GeoTracks):
        # Cut tracks that come much later, i.e. neutron decays
        if track.GetPoint(0)[3] > 1e-5:
            continue

        datum = {}
        particle = track.GetParticle()

        lv = ROOT.TLorentzVector()
        particle.Momentum(lv)
        if lv.E() < (0.5 / 1000):
            continue

        # Note: Will fail if simulation *didn't* run
        # ekin = (lv.E() - particle.GetMass()) * 1000
        # if ekin < 0.1:
        #     continue

        # print(i, particle.GetPdgCode(), ekin, track.GetNpoints())
        datum["pdg"] = particle.GetPdgCode()
        #         datum["ekin"] = ekin

        tr = []
        for n in range(track.GetNpoints()):
            # Check bounds
            if abs(track.GetPoint(n)[2] - (zoffset + 150)) > 300 / 2 * 1.1 or abs(track.GetPoint(n)[1]) > 250 / 2 * 1.0 or abs(track.GetPoint(n)[0]) > 250 / 2 * 1.0:
                continue
            tr.append(
                (
                    track.GetPoint(n)[0] / 10,
                    track.GetPoint(n)[1] / 10,
                    (track.GetPoint(n)[2] - zoffset) / 10,
                )
            )
        datum["track"] = tr
        data.append(datum)
    allparticles.append(data)

    points = []
    for p in e.NeulandPoints:
        pos = p.GetPosition()
        points.append((pos.X() / 10, pos.Y() / 10, (pos.Z() - zoffset) / 10, p.GetLightYield()))
    allpoints.append(points)

    hits = []
    for h in e.NeulandHits:
        pos = h.GetPosition()
        hits.append((pos.X() / 10, pos.Y() / 10, (pos.Z() - zoffset) / 10, h.GetE()))
    allhits.append(hits)

In [7]:
COLORS = palettable.colorbrewer.qualitative.Set1_9.colors
KNOWN_PARTICLES = {
    11: {"name": "$e^-_{\\vphantom{-}}$", "color": COLORS[3], "line": "dashdotted"},
    -11: {"name": "$e^+_{\\vphantom{+}}$", "color": COLORS[3], "line": "dashdotted"},
    -12: {"name": "$\\bar{\\nu}_e$", "color": COLORS[4], "line": "loosely dashdotted"},
    12: {"name": "$\\nu_e$", "color": COLORS[4], "line": "loosely dashdotted"},
    13: {"name": "$\\mu^-$", "color": COLORS[7], "line": "loosely dashdotted"},
    -13: {"name": "$\\mu^+$", "color": COLORS[7], "line": "loosely dashdotted"},
    -14: {"name": "$\\bar{\\nu}_\\mu$", "color": COLORS[4], "line": "loosely dashdotted"},
    14: {"name": "$\\nu_\\mu$", "color": COLORS[4], "line": "loosely dashdotted"},
    22: {"name": "$\\gamma$", "color": COLORS[2], "line": "dotted"},
    111: {"name": "$\\pi^0$", "color": COLORS[5], "line": "loosely dashdotted"},
    211: {"name": "$\\pi^+$", "color": COLORS[5], "line": "loosely dashdotted"},
    -211: {"name": "$\\pi^-$", "color": COLORS[5], "line": "loosely dashdotted"},
    2112: {"name": "Neutron", "color": COLORS[0], "line": "densely dotted"},
    2212: {"name": "Proton", "color": COLORS[1], "line": "solid"},
    1000010020: {"name": "XXX", "color": COLORS[8], "line": "solid"},
    1000010030: {"name": "XXX", "color": COLORS[8], "line": "solid"},
    1000020030: {"name": "XXX", "color": COLORS[8], "line": "solid"},
    1000020040: {"name": "$\\alpha$", "color": COLORS[6], "line": "solid"},
    1000020060: {"name": "XXX", "color": COLORS[8], "line": "solid"},
    1000030060: {"name": "XXX", "color": COLORS[8], "line": "solid"},
    1000030070: {"name": "XXX", "color": COLORS[8], "line": "solid"},
    1000040070: {"name": "XXX", "color": COLORS[8], "line": "solid"},
    1000040090: {"name": "XXX", "color": COLORS[8], "line": "solid"},
    1000040100: {"name": "XXX", "color": COLORS[8], "line": "solid"},
    1000050100: {"name": "XXX", "color": COLORS[8], "line": "solid"},
    1000050110: {"name": "XXX", "color": COLORS[8], "line": "solid"},
    1000060100: {"name": "XXX", "color": COLORS[8], "line": "solid"},
    1000060110: {"name": "XXX", "color": COLORS[8], "line": "solid"},
    1000060119: {"name": "XXX", "color": COLORS[8], "line": "solid"},
    1000060120: {"name": "XXX", "color": COLORS[8], "line": "solid"},
    1000060129: {"name": "XXX", "color": COLORS[8], "line": "solid"},
    1000060130: {"name": "XXX", "color": COLORS[8], "line": "solid"},
    1000070120: {"name": "XXX", "color": COLORS[8], "line": "solid"},
    1000090180: {"name": "XXX", "color": COLORS[8], "line": "solid"},
    1000110230: {"name": "XXX", "color": COLORS[8], "line": "solid"},
    1000120250: {"name": "XXX", "color": COLORS[8], "line": "solid"},
    1000120260: {"name": "XXX", "color": COLORS[8], "line": "solid"},
    1000130260: {"name": "XXX", "color": COLORS[8], "line": "solid"},
    1000130270: {"name": "XXX", "color": COLORS[8], "line": "solid"},
}

In [8]:
LATEX_HEADER = """\
\\documentclass{scrartcl}
\\usepackage{xcolor, tikz}
\\usepackage{pgfplots}
\\pgfplotsset{compat=newest}
\\pagestyle{empty}
"""

LATEX_MID = """\
\\begin{document}
\\begin{tikzpicture}[scale=0.4]
"""

LATEX_END = """\
\\end{tikzpicture}
\\end{document}
"""

AXIS = """\
\\begin{axis}[%
    at={(22.25cm,2cm)}, %4.75
    hide axis,
    scale only axis,
    height=0pt,
    width=0pt,
    colormap={reverse viridis}{
       indices of colormap={
       \\pgfplotscolormaplastindexof{viridis},...,0 of viridis}
    },
    colorbar horizontal,
    point meta min=0,
    point meta max=50,
    label style={font=\\Huge},
    tick label style={font=\\Huge},
    colorbar style={
       width=7cm,
       xtick={50, 25, 0},
    }]
\\end{axis}
"""

In [9]:
def draw_colors(particles):
    out = ""
    pdgs = set([abs(particle["pdg"]) for particle in particles] + [22])
    for pdg in pdgs:
        out += "\\definecolor{pdg%s}{RGB}{%d,%d,%d}\n" % (str(abs(pdg)), *KNOWN_PARTICLES[pdg]["color"])
    return out

In [10]:
def draw_particles(particles):
    out = ""
    for particle in particles:
        out += "\\draw[color=%s, line width=3pt, %s] " % (
            "pdg%s" % str(abs(particle["pdg"])),
            KNOWN_PARTICLES[particle["pdg"]]["line"],
        )
        out += " -- ".join([f"({t[2]}, {t[1]})" for t in particle["track"]])
        out += ";\n"
    return out

In [11]:
def draw_hits(hits):
    out = ""
    colormap = matplotlib.cm.get_cmap("viridis_r")
    emax = 50
    for hit in hits:
        c = colormap(hit[3] / emax)
        out += "\\definecolor{tempcolor}{rgb}{%f,%f,%f}" % (c[0], c[1], c[2])
        out += "\\draw[fill=tempcolor,fill opacity=1] (%f,%f) rectangle (%f,%f);\n" % (
            hit[2] - 0.25,
            hit[1] - 0.25,
            hit[2] + 0.25,
            hit[1] + 0.25,
        )
    return out

In [12]:
def draw_legend(particles):
    out = ""
    pdgs = set([particle["pdg"] for particle in particles] + [22])

    i = 1.
    hasIon = False
    for pdg in pdgs:
        if KNOWN_PARTICLES[pdg]["name"] != "XXX":
            name = KNOWN_PARTICLES[pdg]["name"]
        else:
            if hasIon:
                continue
            else:
                hasIon = True
                name = "Ions"
        # (0.25,%d) -- (1.25,%d)
        out += "\\draw[color=%s, very thick, %s] (22.25,%f) -- (23.25,%f) node [right,black] {\\huge{%s}};\n" % (
            "pdg%s" % str(abs(pdg)),
            KNOWN_PARTICLES[pdg]["line"],
            13.25 - 1.7 * i,
            13.25 - 1.7 * i,
            name,
        )
        i += 1.

    out += AXIS
    return out

In [13]:
def generate_latex_with_legend(en):
    out = LATEX_HEADER
    out += draw_colors(allparticles[en])
    out += LATEX_MID

    # 0, 6
    for i in range(30 - 8, 30):
        out += "\\draw[step=0.5,very thin,gray] (%f,-12.499) grid (%f,0);\n" % (i - 0.001, i + 0.5) #3.25
    # 6, 30
    for i in range(0, 30 - 8):
        out += "\\draw[step=0.5,very thin,gray] (%f,-12.499) grid (%f,12.499);\n" % (i - 0.001, i + 0.5)
    out += "\\draw[very thin,gray] (0,-12.5) -- (30,-12.5) -- (30,12.5) -- (0,12.5);\n"

    out += draw_hits(allhits[en])
    out += draw_particles(allparticles[en])

    out += draw_legend(allparticles[en])

    out += LATEX_END
    return out

In [14]:
def generate_latex_without_legend(en):
    out = LATEX_HEADER
    out += draw_colors(allparticles[en])
    out += LATEX_MID

    for i in range(30):
        out += "\\draw[step=0.5,very thin,gray] (%f,-12.499) grid (%f,12.499);\n" % (i - 0.001, i + 0.5)
    out += "\\draw[very thin,gray] (0,-12.5) -- (30,-12.5) -- (30,12.5) -- (0,12.5);\n"

    out += draw_hits(allhits[en])
    out += draw_particles(allparticles[en])

    out += LATEX_END
    return out

In [15]:
for i in range(40):
    with open(f"results/tracks/tracks{i:02d}.tex", "w") as f:
        f.write(generate_latex_without_legend(i))
    with open(f"results/tracks/tracks{i:02d}l.tex", "w") as f:
        f.write(generate_latex_with_legend(i))

In [16]:
def create_pdf(i, l):
    env = os.environ.copy()
    env["LANG"] = "en_US.UTF-8"
    subprocess.call(["lualatex", f"tracks{i:02d}{l}.tex"], cwd="results/tracks", stdout=subprocess.DEVNULL, env=env)
    subprocess.call(["pdfcrop", f"tracks{i:02d}{l}.pdf", f"tracks{i:02d}{l}.pdf"], cwd="results/tracks", stdout=subprocess.DEVNULL, env=env)
    os.remove(f"results/tracks/tracks{i:02d}{l}.aux")
    os.remove(f"results/tracks/tracks{i:02d}{l}.log")

In [17]:
joblib.Parallel(n_jobs=-1, backend="multiprocessing", verbose=1)(joblib.delayed(create_pdf)(i=i, l=l) for i in range(40) for l in ["", "l"])
print("Done")

[Parallel(n_jobs=-1)]: Using backend MultiprocessingBackend with 32 concurrent workers.


Done


[Parallel(n_jobs=-1)]: Done  80 out of  80 | elapsed:    6.9s finished
